In [ ]:
# CELULA 1: Imports Essenciais
import os
import pandas as pd
import folium # Para exibir o mapa retornado pelas funções de plotagem
from IPython.display import display # Para exibir o mapa no notebook

# IMPORTANTE: Para que este notebook possa chamar funções dos outros .ipynb,
# ou você converte os outros .ipynb em arquivos .py e os importa como módulos,
# ou você copia as definições das funções necessárias diretamente para este notebook.
# Para um projeto de Introdução à IA, copiar as funções necessárias pode ser mais simples.

# === INÍCIO: Funções Copiadas/Adaptadas dos Outros Notebooks ===
# Seria ideal modularizar isso em arquivos .py, mas para simplificar, vamos copiar.

# --- Funções do ProcessaEndereco.ipynb ---
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import openrouteservice # Adicionado aqui pois a função de plotar rota o utiliza

def geocodificar_endereco_brasilia(endereco, tentativas=3, delay=2):
    geolocator = Nominatim(user_agent="projeto_iia_integrado") 
    endereco_completo = f"{endereco}, Brasília, DF, Brazil"
    # print(f"Tentando geocodificar: '{endereco_completo}'") # Removido para diminuir prints
    for tentativa in range(tentativas):
        try:
            location = geolocator.geocode(endereco_completo, timeout=10)
            if location:
                print(f"Endereço encontrado: {location.address}")
                return location.latitude, location.longitude
            else:
                print("Endereço não encontrado.")
        except GeocoderTimedOut:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Timeout. Tentando em {delay}s...")
            time.sleep(delay)
        except GeocoderUnavailable:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Serviço indisponível. Tentando em {delay}s...")
            time.sleep(delay)
        except Exception:
            print(f"Um erro inesperado na geocodificação: {e}")
            return None
    print("Falha na geocodificação.")
    return None

def calcular_distancia_km(coord_usuario, coord_associacao):
    if None in coord_usuario or pd.isna(coord_usuario[0]) or pd.isna(coord_usuario[1]) or \
       None in coord_associacao or pd.isna(coord_associacao[0]) or pd.isna(coord_associacao[1]):
        return float('inf')
    return geodesic(coord_usuario, coord_associacao).km

def carregar_dados_associacoes_geo(caminho_csv_associacoes_geo):
    try:
        df_assoc = pd.read_csv(caminho_csv_associacoes_geo)
        df_assoc['Latitude'] = pd.to_numeric(df_assoc['Latitude'], errors='coerce')
        df_assoc['Longitude'] = pd.to_numeric(df_assoc['Longitude'], errors='coerce')
        df_assoc.dropna(subset=['Latitude', 'Longitude'], inplace=True)
        return df_assoc
    except FileNotFoundError:
        return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

def plotar_mapa_com_rota_para_associacao(
    lat_usuario, lon_usuario, endereco_usuario_popup,
    associacao_selecionada_df_linha, 
    api_key_ors,
    nome_mapa_html="mapa_integrado_recomendacao_final.html",
    pasta_saida_mapa = os.path.join("..", "data", "mapas_finais") 
):
    if lat_usuario is None or lon_usuario is None or associacao_selecionada_df_linha is None or associacao_selecionada_df_linha.empty:
        print("Dados insuficientes para plotar mapa com rota.")
        return None

    assoc_info = associacao_selecionada_df_linha.iloc[0]
    lat_associacao = assoc_info['Latitude']
    lon_associacao = assoc_info['Longitude']
    nome_associacao = assoc_info['Nome_Associacao']
    distancia_reta_km = assoc_info.get('Distancia_km', 0.0)
    produtos_info_popup = assoc_info.get('Produtos_Recomendados_Para_Mostrar', 'N/A') 

    client_ors = openrouteservice.Client(key=api_key_ors)
   
    map_center_lat = lat_usuario 
    map_center_lon = lon_usuario
    zoom_inicial = 12
    if abs(lat_usuario - lat_associacao) > 0.1 or abs(lon_usuario - lon_associacao) > 0.1 : 
        map_center_lat = (lat_usuario + lat_associacao) / 2
        map_center_lon = (lon_usuario + lon_associacao) / 2
        zoom_inicial = 11


    mapa_rota_final = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=zoom_inicial)

    folium.Marker( #icone
        location=[lat_usuario, lon_usuario],
        tooltip="Sua Localização",
        popup=f"<b>Sua Localização</b><br>{endereco_usuario_popup}",
        icon=folium.Icon(color='blue', icon='home', prefix='fa') 
    ).add_to(mapa_rota_final)

    popup_associacao_text = (f"<b>📍 {nome_associacao}</b><br><hr>"
                             f"<b>Distância (reta):</b> {distancia_reta_km:.2f} km<br>")
    
    origem_coords_ors = (lon_usuario, lat_usuario)
    destino_coords_ors = (lon_associacao, lat_associacao)
    rota_geojson = None
    
    try:
        directions = client_ors.directions(
            coordinates=[origem_coords_ors, destino_coords_ors],
            profile='driving-car', format='geojson'
        )
        rota_geojson = directions
        if directions['features'] and directions['features'][0]['properties']['segments']:
            dist_rota_m = directions['features'][0]['properties']['segments'][0]['distance']
            dur_rota_s = directions['features'][0]['properties']['segments'][0]['duration']
            distancia_rota_km_str = f"{dist_rota_m / 1000:.2f} km (aprox. {dur_rota_s / 60:.0f} min)"
            popup_associacao_text += f"<b>Distância (rota):</b> {distancia_rota_km_str}<br>"
    except Exception: # openrouteservice.exceptions.ApiError ou outras
        popup_associacao_text += "<i>Rota pela estrada não pôde ser calculada.</i><br>"

    popup_associacao_text += f"<hr><b>🛍️ Produtos recomendados aqui:</b><br>{produtos_info_popup}"

    folium.Marker(
        location=[lat_associacao, lon_associacao],
        tooltip=f"{nome_associacao} ({distancia_reta_km:.1f} km)",
        popup=folium.Popup(popup_associacao_text, max_width=300),
        icon=folium.Icon(color='green', icon='shopping-basket', prefix='fa')
    ).add_to(mapa_rota_final)

    if rota_geojson and rota_geojson.get('features'):
        folium.GeoJson(
            rota_geojson,
            name=f"Rota para {nome_associacao}",
            style_function=lambda x: {'color': 'red', 'weight': 5, 'opacity': 0.7}
        ).add_to(mapa_rota_final)
    else: 
        folium.PolyLine(
            locations=[(lat_usuario, lon_usuario), (lat_associacao, lon_associacao)],
            color="grey", weight=2, opacity=0.8, dash_array="5, 5",
            tooltip="Linha reta (rota não calculada)"
        ).add_to(mapa_rota_final)
    
    folium.LayerControl().add_to(mapa_rota_final)
    
    os.makedirs(pasta_saida_mapa, exist_ok=True)
    caminho_mapa_final = os.path.join(pasta_saida_mapa, nome_mapa_html)
    mapa_rota_final.save(caminho_mapa_final)
    print(f"-----> Mapa da recomendação final salvo em: {os.path.abspath(caminho_mapa_final)} <-----")
    
    return mapa_rota_final

# functions do RecomendacaoItensKNN.ipynb
from collections import Counter, defaultdict
import numpy as np

def ler_usuarios_compras(caminho_csv_usuarios):
    usuarios_data = {}
    try:
        df_usuarios = pd.read_csv(caminho_csv_usuarios)
        for _, row in df_usuarios.iterrows():
            itens_comprados = set()
            for i in range(1, 6):
                item_col = f'item{i}'
                if pd.notna(row[item_col]):
                    itens_comprados.add(str(row[item_col]).strip())
            usuarios_data[row["nome"]] = {"regiao": row["regiao"], "itens": itens_comprados}
        return usuarios_data
    except FileNotFoundError: return None
    except Exception: return None

def ler_info_produtos_corrigida(caminho_csv_produtos_associacao_simulado):
    info_produtos_agregada = defaultdict(lambda: {
        "organico_geral": False, "sazonalidade": [], "avaliacoes_coletadas": [],
        "associacoes_ofertantes": set(), "avaliacao_media_geral": 3.0
    })
    try:
        df_produtos_assoc = pd.read_csv(caminho_csv_produtos_associacao_simulado)
        for _, row in df_produtos_assoc.iterrows():
            nome_produto = str(row["nome_produto"]).strip()
            if not nome_produto: continue
            if int(row["produto_organico"]) == 1:
                info_produtos_agregada[nome_produto]["organico_geral"] = True
            saz = info_produtos_agregada[nome_produto]["sazonalidade"]
            if int(row["disponivel_primavera"]) == 1 and "primavera" not in saz: saz.append("primavera")
            if int(row["disponivel_verao"]) == 1 and "verao" not in saz: saz.append("verao")
            if int(row["disponivel_outono"]) == 1 and "outono" not in saz: saz.append("outono")
            if int(row["disponivel_inverno"]) == 1 and "inverno" not in saz: saz.append("inverno")
            if pd.notna(row["avaliacao_produto"]):
                try: info_produtos_agregada[nome_produto]["avaliacoes_coletadas"].append(float(row["avaliacao_produto"]))
                except ValueError: pass
            info_produtos_agregada[nome_produto]["associacoes_ofertantes"].add(str(row["nome_associacao"]))
        for produto, dados in info_produtos_agregada.items():
            if dados["avaliacoes_coletadas"]:
                dados["avaliacao_media_geral"] = np.mean(dados["avaliacoes_coletadas"])
        return info_produtos_agregada
    except FileNotFoundError: return None
    except Exception: return None

def calcular_similaridade_usuarios_custom(usuario_alvo_data, outros_usuarios_data):
    similaridades = {}
    itens_alvo = usuario_alvo_data["itens"]
    regiao_alvo = usuario_alvo_data["regiao"]
    for nome_outro, dados_outro in outros_usuarios_data.items():
        intersecao = len(itens_alvo.intersection(dados_outro["itens"]))
        bonus_regiao = 0.5 if dados_outro["regiao"] == regiao_alvo else 0
        similaridades[nome_outro] = intersecao + bonus_regiao
    return sorted(similaridades.items(), key=lambda item: item[1], reverse=True)

def recomendar_itens_knn(
    usuario_alvo_nome, todos_usuarios_data, info_produtos, k_vizinhos=3,
    estacao_atual=None, usuario_prefere_organicos=False
):
    if not todos_usuarios_data or usuario_alvo_nome not in todos_usuarios_data or not info_produtos:
        return [], {}
    usuario_alvo_data = todos_usuarios_data[usuario_alvo_nome]
    outros_usuarios = {n: d for n, d in todos_usuarios_data.items() if n != usuario_alvo_nome}
    usuarios_similares = calcular_similaridade_usuarios_custom(usuario_alvo_data, outros_usuarios)
    vizinhos = [n for n, s in usuarios_similares[:k_vizinhos] if s > 0]
    if not vizinhos: return [], {}

    potenciais = Counter()
    itens_alvo = usuario_alvo_data["itens"]
    for viz in vizinhos:
        for item_v in todos_usuarios_data[viz]["itens"]:
            if item_v not in itens_alvo and item_v in info_produtos:
                p_info = info_produtos[item_v]
                if estacao_atual and p_info.get("sazonalidade") and estacao_atual.lower() not in p_info["sazonalidade"]:
                    continue
                potenciais[item_v] += 1
    if not potenciais: return [], {}

    glob_filtrada = Counter()
    possiveis = set(info_produtos.keys()) - itens_alvo
    for _, d_usr in todos_usuarios_data.items():
        for item_c in d_usr["itens"]:
            if item_c in possiveis and item_c in info_produtos:
                 p_info = info_produtos[item_c]
                 if estacao_atual and p_info.get("sazonalidade") and estacao_atual.lower() not in p_info["sazonalidade"]:
                     continue
                 glob_filtrada[item_c] +=1
    
    mapa_org = {p: d.get("organico_geral", False) for p, d in info_produtos.items()}

    def pontuar(item):
        if item not in info_produtos: return -1
        p_info = info_produtos[item]
        s_viz = potenciais.get(item, 0)
        s_pop = glob_filtrada.get(item, 0)
        s_org = 0
        if mapa_org.get(item, False): 
            s_org = 2
            if usuario_prefere_organicos: s_org += 1
        aval = p_info.get("avaliacao_media_geral", 3.0)
        return (5*s_viz) + (2*s_pop) + (3*s_org) + (1*aval)

    ordenados = sorted(list(potenciais.keys()), key=lambda i: (-pontuar(i), i))
    return ordenados, mapa_org

# funções imoortadas

In [ ]:


def plotar_multiplas_associacoes_mapa(
    lat_usuario, lon_usuario, endereco_usuario_popup,
    df_associacoes_para_plotar, # dataFrame com as top N associações
    nome_mapa_html="mapa_top_recomendacoes.html",
    pasta_saida_mapa=os.path.join("..", "data", "mapas_finais")
):

    if lat_usuario is None or lon_usuario is None:
        print("Coordenadas do usuário inválidas para plotagem.")
        return None
    if df_associacoes_para_plotar is None or df_associacoes_para_plotar.empty:
        print("Nenhuma associação fornecida para plotagem.")
        return None

    # Calcular um ponto centrtl para o mapa (media das coordenadas ou apenas user)
   
    
    map_coords = [[lat_usuario, lon_usuario]]
    for _, row in df_associacoes_para_plotar.iterrows():
        map_coords.append([row['Latitude'], row['Longitude']])
    
    mapa_multiplo = folium.Map(zoom_start=11) # zoom inicial
    
    # Adiciona marcador do usuário
    folium.Marker(
        location=[lat_usuario, lon_usuario],
        tooltip="Sua Localização",
        popup=f"<b>Sua Localização</b><br>{endereco_usuario_popup}",
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    ).add_to(mapa_multiplo)


    for idx, assoc_info in df_associacoes_para_plotar.iterrows():
        popup_text = (
            f"<b>{idx+1}. {assoc_info['Nome_Associacao']}</b><br><hr>"
            f"<b>Localidade:</b> {assoc_info.get('Localidade_Principal_Projeto', 'N/A')}<br>"
            f"<b>Score:</b> {assoc_info.get('Score_Final', 0.0):.2f}<br>"
            f"<b>Distância:</b> {assoc_info.get('Distancia_km', 0.0):.2f} km<br>"
            f"<hr><b>🛍️ Produtos recomendados aqui:</b><br>{assoc_info.get('Produtos_Recomendados_Para_Mostrar', 'N/A')}"
        )
        
        colors = ['green', 'orange', 'purple', 'cadetblue', 'darkred']
        marker_color = colors[idx % len(colors)]

        folium.Marker(
            location=[assoc_info['Latitude'], assoc_info['Longitude']],
            tooltip=f"{assoc_info['Nome_Associacao']} (Score: {assoc_info.get('Score_Final',0.0):.2f})",
            popup=folium.Popup(popup_text, max_width=350),
            icon=folium.Icon(color=marker_color, icon='shopping-basket', prefix='fa')
        ).add_to(mapa_multiplo)
        


    # Ajustar o mapa para mostrar todos os marcadores
    mapa_multiplo.fit_bounds(map_coords)
    
    folium.LayerControl().add_to(mapa_multiplo)

    os.makedirs(pasta_saida_mapa, exist_ok=True)
    caminho_mapa_final = os.path.join(pasta_saida_mapa, nome_mapa_html)
    mapa_multiplo.save(caminho_mapa_final)
    print(f"-----> Mapa com múltiplas associações salvo em: {os.path.abspath(caminho_mapa_final)} <-----")
    
    return mapa_multiplo

In [ ]:
CAMINHO_BASE_DADOS = os.path.join("..", "data") #caminhos

ARQUIVO_USUARIOS = os.path.join(CAMINHO_BASE_DADOS, "usuarios_compras_atualizado.csv")
ARQUIVO_PRODUTOS_ASSOCIACAO = os.path.join(CAMINHO_BASE_DADOS, "associacoes_produtos_simulado.csv") # usar o simulado!
ARQUIVO_ASSOCIACOES_GEO = os.path.join(CAMINHO_BASE_DADOS, "associacoes_geocoords.csv")

API_KEY_ORS = "" 

print("--- Carregando Dados Iniciais ---")
df_usuarios_compras = ler_usuarios_compras(ARQUIVO_USUARIOS)
dict_info_produtos_geral = ler_info_produtos_corrigida(ARQUIVO_PRODUTOS_ASSOCIACAO)
df_associacoes_geo_raw = carregar_dados_associacoes_geo(ARQUIVO_ASSOCIACOES_GEO)

if df_usuarios_compras is None:
    print("ERRO CRÍTICO: Não foi possível carregar os dados de compras dos usuários.")
if dict_info_produtos_geral is None:
    print("ERRO CRÍTICO: Não foi possível carregar as informações dos produtos das associações.")
if df_associacoes_geo_raw.empty:
    print("ERRO CRÍTICO: Não foi possível carregar os dados geográficos das associações.")

print("--- Dados Iniciais Carregados ---\n")

--- Carregando Dados Iniciais ---
--- Dados Iniciais Carregados ---



In [89]:
# CELULA 3: Função Principal de Recomendação Integrada

# No SistemaRecomendacaoIntegrado.ipynb

def obter_recomendacao_integrada(
    nome_usuario_alvo,
    endereco_usuario_alvo,
    estacao_atual_param,
    prefere_organicos_param,
    k_vizinhos_param=5,
    top_n_associacoes_param=1
):
    print(f"\n--- Iniciando Recomendação Integrada para: {nome_usuario_alvo} ---")
    print(f"Endereço: {endereco_usuario_alvo}, Estação: {estacao_atual_param}, Pref. Orgânicos: {prefere_organicos_param}")

    # Valores padrão para retorno em caso de falha
    retorno_falha = pd.DataFrame(), None, None, None

    # Validações iniciais (acessando as variáveis globais carregadas na célula anterior)
    if df_usuarios_compras is None or dict_info_produtos_geral is None or df_associacoes_geo_raw.empty:
        print("ERRO: Dados base (usuários, produtos ou geo associações) não foram carregados corretamente. Abortando.")
        return retorno_falha

    if nome_usuario_alvo not in df_usuarios_compras:
        print(f"ERRO: Usuário '{nome_usuario_alvo}' não encontrado nos dados de compras.")
        return retorno_falha

    # Passo 1: Recomendar Itens (k-NN)
    itens_recomendados_knn, mapa_organicos_dos_itens = recomendar_itens_knn(
        nome_usuario_alvo,
        df_usuarios_compras,
        dict_info_produtos_geral,
        k_vizinhos=k_vizinhos_param,
        estacao_atual=estacao_atual_param,
        usuario_prefere_organicos=prefere_organicos_param
    )

    if not itens_recomendados_knn:
        print(f"AVISO: Nenhum item pôde ser recomendado para '{nome_usuario_alvo}' com os critérios atuais.")
        return retorno_falha
    
    print(f"\nTop itens recomendados (k-NN) para {nome_usuario_alvo}: {itens_recomendados_knn[:5]}")

    # Passo 2: Identificar Associações Candidatas
    associacoes_candidatas_dict = defaultdict(lambda: {"produtos_oferecidos": [], "score_produtos": 0.0})
    for i, item_rec in enumerate(itens_recomendados_knn):
        peso_item = len(itens_recomendados_knn) - i
        if item_rec in dict_info_produtos_geral:
            prod_info = dict_info_produtos_geral[item_rec]
            for nome_assoc in prod_info.get("associacoes_ofertantes", set()):
                org_text = " (Orgânico ✔️)" if prod_info.get("organico_geral") else ""
                aval_text = f"{prod_info.get('avaliacao_media_geral', 0.0):.1f}/5" # Use 0.0 como default se for formatar
                produto_display_info = f"{item_rec}{org_text} [{aval_text}]"
                associacoes_candidatas_dict[nome_assoc]["produtos_oferecidos"].append(produto_display_info)
                associacoes_candidatas_dict[nome_assoc]["score_produtos"] += peso_item * prod_info.get("avaliacao_media_geral", 3.0)

    if not associacoes_candidatas_dict:
        print("AVISO: Nenhuma associação encontrada que oferece os produtos recomendados.")
        return retorno_falha

    df_associacoes_candidatas = pd.DataFrame.from_dict(associacoes_candidatas_dict, orient='index')
    df_associacoes_candidatas.reset_index(inplace=True)
    df_associacoes_candidatas.rename(columns={'index': 'Nome_Associacao'}, inplace=True)

    df_associacoes_candidatas_geo = pd.merge(
        df_associacoes_candidatas,
        df_associacoes_geo_raw[['Nome_Associacao', 'Latitude', 'Longitude', 'Localidade_Principal_Projeto']],
        on='Nome_Associacao',
        how='left'
    ).dropna(subset=['Latitude', 'Longitude'])

    if df_associacoes_candidatas_geo.empty:
        print("AVISO: Associações candidatas não puderam ser geolocalizadas ou não sobraram após merge/dropna.")
        return retorno_falha

    # Passo 3: Geocodificar endereço do usuário e Calcular Distâncias
    coords_usuario = geocodificar_endereco_brasilia(endereco_usuario_alvo)
    if not coords_usuario:
        print(f"ERRO: Não foi possível geocodificar o endereço do usuário: {endereco_usuario_alvo}")
        return retorno_falha # Retorna tupla de falha
    lat_usr, lon_usr = coords_usuario
    print(f"Coordenadas do usuário ({endereco_usuario_alvo}): {lat_usr}, {lon_usr}")

    df_associacoes_candidatas_geo['Distancia_km'] = df_associacoes_candidatas_geo.apply(
        lambda row: calcular_distancia_km((lat_usr, lon_usr), (row['Latitude'], row['Longitude'])),
        axis=1
    )

    # Passo 4: Calcular Score Final de Relevância
    min_dist = df_associacoes_candidatas_geo['Distancia_km'].min()
    max_dist = df_associacoes_candidatas_geo['Distancia_km'].max()
    
    if max_dist == min_dist:
         df_associacoes_candidatas_geo['score_distancia_norm'] = 1.0 if max_dist >=0 else 0.0 # Evita divisão por zero
    else:
        df_associacoes_candidatas_geo['score_distancia_norm'] = df_associacoes_candidatas_geo['Distancia_km'].apply(
            lambda d: (max_dist - d) / (max_dist - min_dist) if (max_dist - min_dist) > 0 else 1.0
        )

    min_score_prod = df_associacoes_candidatas_geo['score_produtos'].min()
    max_score_prod = df_associacoes_candidatas_geo['score_produtos'].max()

    if max_score_prod == min_score_prod:
        df_associacoes_candidatas_geo['score_produtos_norm'] = 1.0 if max_score_prod > 0 else 0.0 # Evita divisão por zero
    else:
        df_associacoes_candidatas_geo['score_produtos_norm'] = df_associacoes_candidatas_geo['score_produtos'].apply(
            lambda s: (s - min_score_prod) / (max_score_prod - min_score_prod) if (max_score_prod - min_score_prod) > 0 else 0.0
        )
    
    peso_distancia = 0.6 
    peso_produtos = 0.4
    df_associacoes_candidatas_geo['Score_Final'] = (
        peso_distancia * df_associacoes_candidatas_geo['score_distancia_norm'] +
        peso_produtos * df_associacoes_candidatas_geo['score_produtos_norm']
    )
    
    df_recomendacoes_finais = df_associacoes_candidatas_geo.sort_values(by='Score_Final', ascending=False)
    
    if df_recomendacoes_finais.empty:
        print("AVISO: Nenhuma associação final após cálculo de score.")
        return retorno_falha

    df_recomendacoes_finais['Produtos_Recomendados_Para_Mostrar'] = df_recomendacoes_finais['produtos_oferecidos'].apply(
        lambda lista_prods: "<br>".join(f"&bull; {p}" for p in lista_prods[:5])
    )

    print("\n--- Associações Finais Recomendadas (ordenadas por score): ---")
    for _, row_assoc in df_recomendacoes_finais.head(top_n_associacoes_param).iterrows(): # Usa top_n_associacoes_param aqui
        print(f"- {row_assoc['Nome_Associacao']} (Score: {row_assoc['Score_Final']:.2f}, Dist: {row_assoc['Distancia_km']:.2f} km)")
        # print(f"  Produtos aqui: {', '.join(row_assoc['produtos_oferecidos'][:3])}...") # Opcional

    return df_recomendacoes_finais.head(top_n_associacoes_param), lat_usr, lon_usr, endereco_usuario_alvo

In [90]:
# CELULA 4: Execução da Recomendação Integrada e Plotagem do Mapa

# --- Parâmetros de Entrada ---
NOME_USUARIO_EXEMPLO = "usuario0007"  # Mude para testar outros usuários
ENDERECO_USUARIO_EXEMPLO = "AV parque aguas claras 3305, Brasília, DF" # Mude para seu endereço ou um de teste
ESTACAO_EXEMPLO = "primavera"  # Pode ser "primavera", "verao", "outono", "inverno", ou None
PREFERE_ORGANICOS_EXEMPLO = True
NUM_ASSOCIACAO_PLOTAR = 2 # Quantas associações mostrar no mapa com rota

# --- Obter Recomendações ---
df_top_associacoes, lat_usr_final, lon_usr_final, endereco_usr_formatado_popup = obter_recomendacao_integrada(
    NOME_USUARIO_EXEMPLO,
    ENDERECO_USUARIO_EXEMPLO,
    ESTACAO_EXEMPLO,
    PREFERE_ORGANICOS_EXEMPLO,
    k_vizinhos_param=5,
    top_n_associacoes_param=NUM_ASSOCIACAO_PLOTAR 
)

if lat_usr_final is not None and not df_top_associacoes.empty:
     
    associacao_para_plotar_df = df_top_associacoes.head(2) # Pega a primeira da lista ordenada
    
    print(f"\nPlotando mapa com rota para a principal associação recomendada: {associacao_para_plotar_df.iloc[0]['Nome_Associacao']}")
    
    mapa_gerado = plotar_mapa_com_rota_para_associacao(
        lat_usr_final,
        lon_usr_final,
        endereco_usr_formatado_popup, # endereço original para o popup
        associacao_para_plotar_df, # dataFrame de uma linha
        API_KEY_ORS, 
        nome_mapa_html=f"mapa_recomendacao_{NOME_USUARIO_EXEMPLO}.html"
    )
    if mapa_gerado:
        display(mapa_gerado)
else:
    if lat_usr_final is None:
        print("\nNão foi possível obter coordenadas do usuário para plotar o mapa.")
    elif df_top_associacoes.empty:
        print("\nNenhuma associação para plotar no mapa.")


--- Iniciando Recomendação Integrada para: usuario0007 ---
Endereço: AV parque aguas claras 3305, Brasília, DF, Estação: primavera, Pref. Orgânicos: True

Top itens recomendados (k-NN) para usuario0007: ['Abacate', 'Jiló', 'Couve', 'Pepino']
Coordenadas do usuário (AV parque aguas claras 3305, Brasília, DF): -15.8355997, -48.0339063

--- Associações Finais Recomendadas (ordenadas por score): ---
- Associação dos Produtores da Agricultura Familiar (Aspaf) (Score: 0.96, Dist: 5.73 km)
- Cooperativa Agrícola da Região de Planaltina (Cootaquara) (Score: 0.82, Dist: 8.95 km)

Plotando mapa com rota para a principal associação recomendada: Associação dos Produtores da Agricultura Familiar (Aspaf)
-----> Mapa da recomendação final salvo em: c:\Users\marce\ProjetoIIA\data\mapas_finais\mapa_recomendacao_usuario0007.html <-----


In [ ]:
# CELULA NOVA: Demonstração - Plotar as Top 3 Associações Recomendadas

# --- Parâmetros de Entrada para a Demonstração ---
NOME_USUARIO_TOP3 = "usuario0111"  # Mude para testar outros usuários
ENDERECO_USUARIO_TOP3 = "taguatinga, Brasília, DF" 
ESTACAO_TOP3 = "outono"  # P "primavera", "verao", "outono", "inverno", ou None
PREFERE_ORGANICOS_TOP3 = True
NUM_ASSOCIACAO_PARA_SCORE = 5 # n associações queremos que o score retorne
K_VIZINHOS_KNN_TOP3 = 5

print(f"\n\n--- DEMONSTRAÇÃO: OBTENDO TOP {NUM_ASSOCIACAO_PARA_SCORE} RECOMENDAÇÕES ---")
df_top_N_associacoes, lat_usr_top3, lon_usr_top3, endereco_usr_formatado_top3 = obter_recomendacao_integrada(
    NOME_USUARIO_TOP3,
    ENDERECO_USUARIO_TOP3,
    ESTACAO_TOP3,
    PREFERE_ORGANICOS_TOP3,
    k_vizinhos_param=K_VIZINHOS_KNN_TOP3,
    top_n_associacoes_param=NUM_ASSOCIACAO_PARA_SCORE 
)

if lat_usr_top3 is not None and not df_top_N_associacoes.empty:
    print(f"\nPlotando mapa com as top {len(df_top_N_associacoes)} associações recomendadas...")
    
    mapa_gerado_multiplo = plotar_multiplas_associacoes_mapa(
        lat_usr_top3,
        lon_usr_top3,
        endereco_usr_formatado_top3, 
        df_top_N_associacoes,        
        nome_mapa_html=f"mapa_top{NUM_ASSOCIACAO_PARA_SCORE}_recomendacoes_{NOME_USUARIO_TOP3}.html"
        
    )
    if mapa_gerado_multiplo:
        display(mapa_gerado_multiplo)
else:
    if lat_usr_top3 is None:
        print("\nNão foi possível obter coordenadas do usuário para plotar o mapa das top N.")
    elif df_top_N_associacoes.empty:
        print("\nNenhuma associação para plotar no mapa das top N.")



--- DEMONSTRAÇÃO: OBTENDO TOP 5 RECOMENDAÇÕES ---

--- Iniciando Recomendação Integrada para: usuario0111 ---
Endereço: taguatinga, Brasília, DF, Estação: outono, Pref. Orgânicos: True

Top itens recomendados (k-NN) para usuario0111: ['Cenoura', 'Abóbora', 'Banana', 'Agrião', 'Couve']
Coordenadas do usuário (taguatinga, Brasília, DF): -15.8335277, -48.0565716

--- Associações Finais Recomendadas (ordenadas por score): ---
- Associação Agrícola do Distrito Federal e Ride (Agrifam) (Score: 0.92, Dist: 0.04 km)
- Cooperativa Mista da Agricultura Familiar do Meio Ambiente e da Cultura do Brasil (Coopbrasil) (Score: 0.77, Dist: 20.27 km)
- Associação dos Trabalhadores Rurais da Agricultura Familiar do Assentamento Chapadinha (Astraf) (Score: 0.76, Dist: 7.95 km)
- Cooperativa dos Produtores Rurais de Planaltina de Goiás e Região (Prorural) (Score: 0.69, Dist: 19.18 km)
- Cooperativa Agrícola da Região de Planaltina (Cootaquara) (Score: 0.59, Dist: 6.53 km)

Plotando mapa com as top 5 asso